In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import hvpy
import astropy.units as u

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.timeseries import BinnedTimeSeries
from astropy.timeseries import TimeSeries

In [ ]:
from scipy import ndimage
from skimage import measure

In [ ]:
from sunpy.map import Map
from sunpy.coordinates import frames
from sunpy.map.maputils import all_coordinates_from_map, coordinate_is_on_solar_disk

In [ ]:
def main(time): # function that receives as a parameter the time series to be analyzed
    
    Day = []   # python list that receive time objects from maps
    Latitudes = [] # python list that receives the latitudes of the sunspots of the respective time objects
    Areas = [] # python list that receives the areas of the spots of the respective time objects
    Number_of_Sunspot_Groups = []  # python list that receives the number of the spots groups of the respective time objects
     
    for i in range(len(time)): # repeating loop to open images day by day
        day = time[i][0]
        print(day)
        flaws = Time(['1998-11-18T12:00:00', '1998-11-19T12:00:00', '1998-11-20T12:00:00', '1998-11-21T12:00:00',
                      '2002-08-26T12:00:00', '2004-03-09T00:00:00','2005-07-17T12:00:00', '2008-04-26T12:00:00', 
                      '2008-08-03T12:00:00', '2009-03-28T12:00:00', '2010-03-08T12:00:00'])
        
        try: # The exception for error is due to some images with capture errors present in the sasos database 
            # or internet connection problems when running the code. This way, progress is not lost. 
            # It is recommended to test the code without the exession to verify that it works correctly.
        # the dates above are dates in which the images have some type of capture anomaly, which impairs the detection of spots. We chose to remove them

            if day in flaws: # If the analysis date is in the list of defective dates, only the date and missing data for latitude, area and sunspot group are added to the output data.
                Day.append(day.value)
                Latitudes.append(np.NaN)
                Areas.append(np.NaN)
                Number_of_Sunspot_Groups.append(0)

            else: # Otherwise, the algorithm follows
                
                map_file =  get_map(day) # Create the coordinate map
                tophat_map = top_hat(day, map_file) # Apply the transform
                erode_map = erode(day, tophat_map) # Apply erosion to the transformed image
                sun_radius = get_sun_radius(map_file) # get the sun's ray in pixels
                contours = get_contours(tophat_map) # contours to sunspots
                group_contours = get_contours(erode_map) # contours to sunspots groups


                if len(contours) == 0:  # If the image has no sunspots, add missing data to the algorithm output
                    Day.append(map_file.date.value)
                    Latitudes.append(np.NaN)
                    Areas.append(np.NaN)

                else: # If it has sunsspots, it obtains the latitudes, longitudes and areas of each one.
                    latitudes, longitudes, Am = get_coord_and_area(contours, tophat_map, sun_radius)

                    for j in range(len(latitudes)):
                        Day.append(map_file.date.value)
                        Latitudes.append(latitudes[j])
                        Areas.append(Am[j])
                        
                if len(group_contours) == 0:  # If the image has no sunspots, add missing data to the algorithm output
                    Number_of_Sunspot_Groups.append(0)
                else: # If it has sunspots, it obtains the number of sunspot groups
                    Number_of_Sunspot_Groups.append(len(contours))
        except:
            Day.append(Time(day))
            Latitudes.append(np.NaN)
            Areas.append(np.NaN)
            Number_of_Sunspot_Groups.append(0)
            
    return Day, Latitudes, Areas, Number_of_Sunspot_Groups

In [ ]:
def get_map(day):
    
    if day < Time('2010-12-06'):  
        s_id = hvpy.DataSource.MDI_INT.value # for dates before 12/06/2010, we must use the MDI instrument
    else:
        s_id= hvpy.DataSource.HMI_INT.value # for dates after 12/06/2010, we must use the HMI instrument
        
    hmi_file = hvpy.save_file(hvpy.getJP2Image(day.datetime, s_id), "T.JPEG2000",  overwrite=True) # get the jpeg2000 file of the base of the Helioview
    map_file = Map(hmi_file) # Create a coordinate map from jpg2000 metadata using Sunpy
    
    return map_file

In [ ]:
def top_hat(day, map_file):
    
    pixel_matrix = 255 - map_file.data # create negative image

    if day < Time('2010-12-06'):
        pixel_matrix = cv.medianBlur(pixel_matrix, 5) # Filter to enhance the transform
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(45,45)) #  define the size and type of structuring element used
    else:
        pixel_matrix = cv.medianBlur(pixel_matrix,15) 
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(135,135))
    
    tophat = cv.morphologyEx(pixel_matrix, cv.MORPH_TOPHAT, kernel) # Apply the transform
    _, binary = cv.threshold(tophat,15,255,cv.THRESH_BINARY_INV) # Apply the binarization
    tophat_map = Map(binary, map_file.meta) # Creates a new coordinate map
    
    return tophat_map

In [ ]:
def get_sun_radius(map_file):
    
    # In this part, we create a Boolean mask, where the true values indicate the solar disk 
    # from that, we outline the entire disk and obtain the regions outlined in x and y 
    # after that, we subtract the end of the x region from the beginning of the x region and obtain the diameter of the sun 
    # and dividing the diameter by 2 we get the sun's radius in pixels.
    
    hpc_coords = all_coordinates_from_map(map_file)
    mask = coordinate_is_on_solar_disk(hpc_coords)
    cont_sun = measure.find_contours(mask, 1)
    labeled_mask, num_labels = ndimage.label(mask)
    regions = ndimage.find_objects(labeled_mask)

    for r in regions:
        dy, dx = r
        radius = (dx.stop - dx.start)/2
    
    return radius

In [ ]:
def erode(day, map_file):
    pixel_matrix = map_file.data 

    if day < Time('2010-12-06'):
        pixel_matrix = cv.medianBlur(pixel_matrix, 5) # Filter to enhance the transform
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(45,45)) #  define the size and type of structuring element used
    else:
        pixel_matrix = cv.medianBlur(pixel_matrix,15) 
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(135,135))
    
    erode = cv.erode(pixel_matrix, kernel, iterations=1) # Image erosion for joining sunspots into groups
    erode_map = Map(erode, map_file.meta) # Creating a  Erode map
    
    return erode_map

In [ ]:
def get_contours(tophat_map): # In this part, we outline the sunspots present in the transformed image

    threshold = 0
    binary_image = tophat_map.data == threshold
    contours, hierarchy = cv.findContours(binary_image.astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    return contours

In [ ]:
def get_coord_and_area(contours, tophat_map, sun_radius):
    posição_x = []
    posição_y = []
    areas_pixels = []
    
    # In this part, we calculate the area in pixels of the contours found in the previous function, as well as the sunspot centroid
    
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 16:
            areas_pixels.append(area)
            M = cv.moments(cnt)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            posição_x.append(cx)
            posição_y.append(cy)
    
        # In this part, we convert the position of sunspot centroids to stonyhurts coordinates
        
        coord = tophat_map.pixel_to_world(posição_x*u.pix, posição_y*u.pix)
        x_arc, y_arc = coord.Tx, coord.Ty

        c = SkyCoord(x_arc.value*u.arcsec, y_arc.value*u.arcsec, frame=frames.Helioprojective, obstime=tophat_map.date,
                        observer="earth")
        c = c.transform_to(frames.HeliographicStonyhurst)

        long = c.lon.value
        lat = c.lat.value

        latitudes = [La for La in lat]
        longitudes = [Lo for Lo in long]
        
        # In this part, we convert the area of sunspots from pixels to millionths of a solar hemisphere
        
        Am = [(As*10**6)/(2*np.pi*sun_radius**2*np.cos(np.radians(Ro))) for As, Ro in zip(areas_pixels, longitudes)]
        
    return latitudes, longitudes, Am

In [ ]:
time = BinnedTimeSeries(time_bin_start= '1998-01-01T18:00:00', 
                       time_bin_size=1 * u.d, n_bins=9709)  # Time series containing the initial observation bin and the number of bins to be observed

In [ ]:
if __name__ == '__main__':
    Day, Latitudes, Areas, Number_of_Sunspot_Groups = main(time) 

In [ ]:
# After the code is finished, two csv files are generated. One with individual sunspots and the other with the number of groups of spots per day

In [ ]:
lista = [Time(i) for i in Day]
Day = lista

In [ ]:
ts1 = TimeSeries(time = Day)

ts1['Lat'] = Latitudes
ts1['Area'] = Areas

df1 = ts1.to_pandas()
df1.to_csv('Sunspots.csv')

In [ ]:
ts2 = time

ts2['Number of Sunspot_Groups'] = Number_of_Sunspot_Groups
df2 = ts2.to_pandas()
df2.to_csv('Groups.csv')